# `nu18` vs `n24`

`n24` is the "Number of children eligible for Child Tax Credit" according to [Tax-Calculator documentation](http://open-source-economics.github.io/Tax-Calculator/). `nu18` is the number of people under age 18.

Based on the Child Tax Credit's [definition of child](https://www.thebalance.com/child-tax-credit-3193009), this should be a subset of children under age 18. One criterion is being under age 17, and others limit further within that.

This notebook examines tax units that violate this assumption by having `n24 > nu18`. This is discussed in [taxdata issue #157](https://github.com/open-source-economics/taxdata/issues/157).

*Data: CPS  |  Tax year: 2014  |  Author: Max Ghenis  |  Date run: 2018-02-28*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [2]:
tc.__version__

'0.16.2'

In [3]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Show one decimal in tables.
pd.set_option('precision', 2)

## Summaries

In [6]:
recs = tc.Records.cps_constructor()
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


We only care about records `n24 > 0`.

In [7]:
full = calc.dataframe(['s006', 'nu18', 'n24'])

In [8]:
full['nu18_s006'] = full['nu18'] * full['s006']
full['n24_s006'] = full['n24'] * full['s006']
full['n24_gt_nu18'] = full['n24'] > full['nu18']
full['n24_vs_nu18'] = np.where(full['n24'] > full['nu18'], 'n24 greater',
                               np.where(full['nu18'] > full['n24'], 
                                        'nu18 greater', 'equal'))

Total `nu18` and `n24`.

In [9]:
full['n24_s006'].sum() / 1e6

81.645326520011665

In [10]:
full['nu18_s006'].sum() / 1e6

78.788336770005856

In [11]:
full.pivot_table(index='n24_vs_nu18', 
                 values=['s006', 'n24_s006', 'nu18_s006'],
                 aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_vs_nu18,,,
equal,5.98e+07,5.98e+07,1.44e+08
n24 greater,1.57e+07,1.44e+06,9.05e+06
nu18 greater,6.11e+06,1.75e+07,9.77e+06


Limit to `n24>0` for remainder.

In [12]:
df = full[full['n24'] > 0]

In [13]:
df.pivot_table(index='n24_gt_nu18', 
               values=['s006', 'n24_s006', 'nu18_s006'],
               aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_gt_nu18,,,
False,6.60e+07,6.97e+07,3.74e+07
True,1.57e+07,1.44e+06,9.05e+06


Drill into records where `n24>nu18` by `nu18`.

In [14]:
df[df['n24_gt_nu18']].pivot_table(
    index='nu18', 
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

,n24_s006,nu18_s006,s006
nu18,,,
0.0,1.22e+07,0.00e+00,7.71e+06
1.0,3.24e+06,1.28e+06,1.28e+06
2.0,9.95e+04,5.54e+04,2.77e+04
3.0,5.37e+04,3.93e+04,1.31e+04
4.0,8.80e+04,7.04e+04,1.76e+04


In [15]:
df[df['n24_gt_nu18']].pivot_table(
    index='nu18', 
    columns='n24',
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

n24_s006                                           nu18_s006             \
n24        1.0       2.0       3.0        4.0        5.0       1.0        2.0   
nu18                                                                            
0.0   4.57e+06  4.31e+06  2.10e+06  860089.17  374394.73       0.0       0.00   
1.0        NaN  1.64e+06  8.61e+05  478895.03  253801.17       NaN  821119.77   
2.0        NaN       NaN  5.04e+04   21357.81   27727.52       NaN        NaN   
3.0        NaN       NaN       NaN   47190.87    6508.30       NaN        NaN   
4.0        NaN       NaN       NaN        NaN   88014.73       NaN        NaN   

                                          s006                       \
n24         3.0        4.0       5.0       1.0       2.0        3.0   
nu18                                                                  
0.0        0.00       0.00      0.00  4.57e+06  2.16e+06  699162.46   
1.0   287157.91  119723.76  50760.23       NaN  8.21e+05  287157.91   
2.0    33626.02   10678.91  11091.01       NaN       NaN   16813.01   
3.0         NaN   35393.15   3904.98       NaN       NaN        NaN   
4.0         NaN        NaN  70411.79       NaN       NaN        NaN   

                           
n24         4.0       5.0  
nu18                       
0.0   215022.29  74878.95  
1.0   119723.76  50760.23  
2.0     5339.45   5545.50  
3.0    11797.72   1301.66  
4.0         NaN  17602.95

Share of tax units with `n24>0` where `n24>nu18`.

In [16]:
df.loc[df['n24_gt_nu18'], 's006'].sum() / df['s006'].sum()

0.19482597729249573